## Convolutional Neural Network (CNN)
Convolutional Neural Network (CNN)은 deep learning의 한 계열로, 데이터의 공간적 배치와 지역적인 특징을 잡아내어 주기 때문에 이미지 인식 등의 분야에서 폭넓게 활용되고 있다. 이번 장에서는 앞서 실습한 MNIST 데이터셋을 다시 이용하여, 이번에는 CNN을 통해 정확도를 높이는 과정을 연습해 보자.

* 본 실습의 예제는 https://www.tensorflow.org/get_started/ 와 https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist/#0에서 나온 예제들을 수정하여 만들어졌습니다.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow as tf

In [5]:
X = tf.placeholder(tf.float32, shape=[None, 784], name='image')
X_image = tf.reshape(X, [-1,28,28,1])

Y = tf.placeholder(tf.float32, shape=[None, 10], name='label')

앞서와 마찬가지로 X와 Y를 placeholder로 지정하는 데, 앞서와는 다르게 image를 담을 X의 크기를 [,784]에서 tf.reshape을 이용하여 [,28,28,1]로 바꾸어 주었다. 이는 본래의 이미지 크기인 28x28의 크기로 다시 되돌리는 것이며, 끝의 1은 이미지가 흑백 이미지이기 때문이다. 컬러 이미지의 경우 RGB-coding을 통해 수가 3이 된다. 이와 같이 CNN에서는 이미지 본래의 사이즈를 사용하기 때문에 공간적인 특성들을 파악할 수 있게 된다.

CNN은 input layer - (convolution layer - pooling layer - convolution layer - pooling layer - ...  반복) - fully-connected layer - output layer와 같은 형태로 구성된다. 본 예제에서는 convolution - pooling이 두 번 쌓여 있는 구조의 network를 만들자. 각각의 layer를 만들기 위해서는 weight W와 bias b의 크기를 지정해 주어야 하고, 그들을 통해 어떤 형태의 convolution과 pooling을 할 것인지 parameter를 정해줘야 한다. 우선 weight와 bias를 생성하는 함수들을 정의하자. 반복되는 작업이기 때문에 함수를 쓰는 것이 편리하다.

In [12]:
def weight_variable(shape, variable_name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=variable_name)

def bias_variable(shape, variable_name):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name=variable_name)

weight_variable 함수는 크기 shape과 이름 variable_name을 인자로 받아서, 해당하는 크기와 이름을 가진 weight variable을 생성한다. 이 때 initial value로서 tf.truncated_normal을 사용해서 표준편차 0.1인 정규분포로부터 랜덤하게 초기값들을 생성했다. 앞서의 경우처럼 0으로 초기화할 경우 학습이 잘 일어나지 않는 문제가 있다. 

bias_variable 함수 역시 마찬가지 방법으로 bias variable을 생성하지만, 초기값으로 0.1로 통일시켰다. 이는 뒤에서 설명할 activation function인 ReLU (rectified linear unit)가 음수일 때는 0의 값을 가지므로, 학습이 잘 일어나게 하기 위해서는 양의 값을 가지도록 유도하는 것이 좋기 때문이다.

다음에는 weight, bias를 입력 받아서 각각 convolution layer와 pooling layer를 생성하는 함수를 정의하자.

In [13]:
def convolution(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def pooling(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

convolution 함수는 바로 앞 layer의 cell들의 output x를 받아, weight W로 convolution operation을 한 결과를 돌려 준다. W의 경우 convolution kernerl 혹은 filter로 생각하면 된다. 즉, W의 크기를 가진 filter가 x라는 이미지 위를 stride의 크기만큼 차례로 움직여 가면서 convolution (이미지 픽셀들과 weight의 값들의 곱하고 더하는 operation)한 결과를 다시 새로운 이미지로 출력하는 것이다. 

tf.nn.conv2d의 인자들 중에서 strides=[1,1,1,1]의 의미는 각 차원으로 1씩 convolution kernel을 이동해 가면서 계산하라는 의미이다. 데이터가 흑백 이미지이므로 실제로는 가로 세로 1씩 움직여 가면서 convolutuon을 한다. 이 경우 경계 부분에 와서는 kernel과 매칭되는 데이터가 없는 구간이 나오는데, 이때 그 구간에 값을 채워서 계산하기 위한 것을 padding이라 한다. stride의 사이즈가 1이고 padding을 하기 때문에, convolution의 결과물은 입력 이미지와 크기가 같다.

pooling 함수는 convolution의 결과로 나온 새로운 이미지를, 어떤 특정 영역 안에서 평균이나 최대값 등의 대표값을 뽑아 내어 그 값으로 새로운 이미지를 만드는 역할을 한다. tf.nn.max_pool은 이름대로 max pooling, 즉 최대값을 뽑아내는 함수이며, ksize를 통해 커널의 크기를, strides를 통해 stride의 크기를 정한다. 이 예제에서는 2x2의 크기를 가진 커널을 가로 세로 2칸씩 움직여서 max pooling을 한다. 역시 경계부분에 와서는 same padding을 적용한다. stride가 2이므로 그 결과물은 입력 이미지보다 가로와 세로가 각각 절반의 크기를 가진다. 

이제 위의 함수들을 사용하여 실제로 2쌍의 convolution-pooling layer 층을 만들어 보자.

In [25]:
W_conv1 = weight_variable([5, 5, 1, 32], 'W_conv1')
b_conv1 = bias_variable([32], 'b_conv1')

h_conv1 = tf.nn.relu(convolution(X_image, W_conv1) + b_conv1)
h_pool1 = pooling(h_conv1)

In [26]:
W_conv2 = weight_variable([5, 5, 32, 64], 'W_conv2')
b_conv2 = bias_variable([64], 'b_conv2')

h_conv2 = tf.nn.relu(convolution(h_pool1, W_conv2) + b_conv2)
h_pool2 = pooling(h_conv2)

위 두 블락의 코드 각각이 1층의 hidden layer의 convolutuion-pooling 쌍, 2층의 convolution-pooling 쌍을 만든다. 이때 유의해서 볼 점은 weight와 bias의 크기이다. W_conv1은 [5,5,1,32]의 크기를 가졌는데, 그 의미는 5x5의 크기를 가진 2차원 kernel이고 (세번째 수가 1이므로), 마지막 숫자 32는 결과물로 나온 이미지의 갯수이다. 즉 1개의 이미지당 32개의 convolution된 이미지를 만들어 낸다. 따라서 bias의 크기도 [32]가 된다. 

h_conv1은 그렇게 해서 나온 결과물(hidden-layer)이고, h_pool1은 이를 다시 pooling해서 나온 결과이다. pooling의 결과 h_pool1의 크기는 [None, 14, 14, 32]가 된다. 크기가 14x14가 된것은 앞서 pooling에서 stride가 가로 세로 2씩이었기 때문에 각 차원에서 크기가 반으로 줄어들었다.

W_conv2는 다시 32개 층의 이미지 (h_pool1)을 받아서 convolution하여 64개의 이미지를 만들기 위해 크기가 설정되었다. h_conv2즌 h_pool1과 W_conv2를 convolution한 결과이며, 여기에 다시 pooling을 하여 h_pool2가 만들어진다. h_pool2의 크기는 다시 가로 세로가 반씩 줄어들어 [None, 7, 7, 64]가 되었다.

이렇게 두 번의 convolution을 거친 후, 그 결과를 이번에는 1024개의 뉴런을 가진 fully-connected layer로 전달한다.

In [28]:
W_fc1 = weight_variable([7 * 7 * 64, 1024], 'W_fc1')
b_fc1 = bias_variable([1024], 'b_fc1')

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

위의 코드에서 W_fc1은 [7*7*64, 1024]의 크기를 가지고 있는 데, 이는 h_pool2가 현재는 3차원 텐서의 크기 [7,7,64]를 가졌지만, fully-connected layer와 연결하기 위해서는 이를 다시 1차원 텐서로 길게 늘여뜨려야 하기 때문이다. 모든 원소를 보존하기 위해 h_pool2_flat이라는 layer가 tf.reshape을 통해 크기가 재구성되고, W_fc1와의 곱을 통해 1024개의 길이를 가진 1차원 tensor로 변환된다. 이 layer의 output은 tf.nn.relu라는 activation function을 거쳐서 나오게 되어 h_fc1이 된다. 

이대로 이어서 그 결과를 최종 output layer에 연결해도 되지만, 학습에서 overfitting이 일어나는 것을 막기 위해서 dropout이라는 테크닉을 적용하자. dropout은 tf.nn.dropout을 통해 매번 학습때마다 랜덤하게 fully-connected layer에서 일정 비율의 neuron들을 일시적으로 제거하여 계산한다. 

In [29]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

이제 우리의 CNN 구성의 마지막 단계에 왔다. 앞에서 dropout을 통해서 만들어진 h_fc1_drop layer와 output을 위한 W_fc2 weight를 곱하고 b_fc2를 더해서 최종적으로 10개의 neuron을 가진 output layer인 y_conv를 다음과 같이 계산한다.

In [32]:
W_fc2 = weight_variable([1024, 10], 'W_fc2')
b_fc2 = bias_variable([10], 'b_fc2')

Y_prediction = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

앞 장에서와 마찬가지 훈련을 위해 loss function을 cross-entropy의 평균값으로 정의하고, 이를 최소화하기 위한 optimizer를 세팅한다.

In [33]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, 
                                                        logits=Y_prediction)
loss = tf.reduce_mean(cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

tf.nn.softmax_cross_entropy_with_logits는 cross-entropy를 계산해 주며, 이의 평균이 loss로 정의된다. train_step은 앞서와 달리 Adam optimizer를 사용해서 loss를 최소화한다. 

앞서와 마찬가지로 accuracy를 계산하기 위한 코드를 만든다.

In [35]:
correct_prediction = tf.equal(tf.argmax(Y_prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

이제 세션을 생성하고, variable을 초기화 한 다음에 실제로 학습과정을 돌려보자. 2000번의 반복과 한 번에 50개의 데이터를 가진 batch를 만들어서 입력하자. 시간이 다소 걸리므로 진행상황을 알아보기 위해 매 100번의 반복마다 training accuracy를 출력하는 명령문을 넣어 주자.

In [39]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        X:batch[0], Y: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={X: batch[0], Y: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.14
step 100, training accuracy 0.9
step 200, training accuracy 0.94
step 300, training accuracy 0.9
step 400, training accuracy 0.94
step 500, training accuracy 0.94
step 600, training accuracy 0.92
step 700, training accuracy 0.98
step 800, training accuracy 0.98
step 900, training accuracy 1
step 1000, training accuracy 0.98
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.94
step 1500, training accuracy 0.96
step 1600, training accuracy 0.96
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 1
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 1
step 2400, training accuracy 0.98
step 2500, training accuracy 0.94
step 2600, training accuracy 1
step 2700, training accuracy 0.96
step 2800, training accuracy 0.98
step 2900, training accuracy 1
step 3000, trainin

위의 훈련과정에서 중간에 training accuracy를 출력할 때는 keep_prob 인자의 값이 1.0이고, 실제 훈련을 하는 train_step에서는 keep_prob 인자의 값이 0.5임에 주목하자. 이는 dropout 방법에서 훈련 도중에는 fully-connected network의 절반에 해당하는 neuron들만을 랜덤하게 업데이트 하다가, 실제 평가 시에는 그 neuron들 전부를 사용한다는 의미이다.

훈련이 끝났으면 feed_deict에 test data를 입력하여 test accuracy를 출력해 보자.

In [40]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9932


test acuuracy가 99%가 넘는 좋은 결과를 보여 주는 것을 확인할 수 있다. 